In [2]:
import geopandas as gpd
import pandas as pd
import folium

# Read the hospital and sidewalk data
hospitals = gpd.read_file("../data/Hospitals.geojson")
sidewalks = gpd.read_file("../data/Sidewalks.json")

# Convert the hospital geometry points to circle areas (buffers) with a 100-meter radius
hospitals["buffer"] = hospitals.geometry.buffer(100)

# Set the hospital geometry to the buffer column
hospitals.set_geometry("buffer", inplace=True)

# Perform a spatial join between the hospitals' buffers and the sidewalks
sidewalks_within_buffer = gpd.sjoin(sidewalks, hospitals, op="intersects", how="inner")

# Group the sidewalks by the hospital's ID and calculate the average score
avg_score_by_hospital = sidewalks_within_buffer.groupby("name")["score_1"].mean().reset_index()

# Merge the average scores with the hospital data
result = hospitals.merge(avg_score_by_hospital, left_on="name", right_on="name")
print(result)







C:\Users\xiao\AppData\Local\Temp\ipykernel_2412\1374139438.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hospitals["buffer"] = hospitals.geometry.buffer(100)
c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\xiao\AppData\Local\Temp\ipykernel_2412\1374139438.py:16: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:6492
Right CRS: EPSG:4326

  sidewalks_within_buffer = gpd.sjoin(sidewalks, hospitals, op="intersects", how="inner")


KeyError: 'name'

In [ ]:
# Display the map
m = folium.Map(location=[result.geometry.centroid.y.mean(), result.geometry.centroid.x.mean()], zoom_start=12)

# Add the hospital markers to the map
for idx, row in result.iterrows():
    folium.Marker(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        tooltip=row["name"],
        icon=folium.Icon(color="red", icon="hospital", prefix="fa")
    ).add_to(m)
    
# Add the sidewalk buffers to the map
for idx, row in sidewalks_within_buffer.iterrows():
    folium.Circle(
        location=[row.geometry.centroid.y, row.geometry.centroid.x],
        radius=50,
        tooltip=f"Score: {row['score']}",
        color="blue",
        fill=False
    ).add_to(m)

# Save and display the map
m.save("./HospitalsAverageScoremap.html")
